<a href="https://colab.research.google.com/github/luck058/yfinance-prediction/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>